## Problem: 

An airline wants to know the minimum number of flight routes to support such that any given airport can connect to any other airport.

Given the following 3 inputs ...
- a list of one way airport routes [origin, destination] --> <b><i>routes</i></b>
- list of all airports --> <b><i>airports</i></b>
- a starting airport --> <b><i>startingAirport</b></i><br>

write the function <u><b>minAdditionalRoutes</u></b> that returns the <u>minimum</u> number of additional routes to add to all one way<b><i>routes</i></b> such that the <b><i>startingAirport</b></i> can travel to <u>any</u> any other airport in the list <b><i>airports</i></b>.  

In [334]:
routes = [
    ['DSM', 'ORD'],
    ['ORD', 'BGI'],
    ['BGI', 'LGA'],
    ['SIN', 'CDG'],
    ['CDG', 'SIN'],
    ['CDG', 'BUD'],
    ['DEL', 'DOH'],
    ['DEL', 'CDG'],
    ['TLV', 'DEL'],
    ['EWR', 'HND'],
    ['HND', 'ICN'],
    ['HND', 'JFK'],
    ['ICN', 'JFK'],
    ['JFK', 'LGA'],
    ['EYW', 'LHR'],
    ['LHR', 'SFO'],
    ['SFO', 'SAN'],
    ['SFO', 'DSM'],
    ['SAN', 'EYW']
    #,['LGA', 'EWR']
    #,['LGA', 'TLV']
]


airports = ['BGI', 'CDG', 'DEL', 'DOH', 'DSM', 'EWR', 'EYW', 'HND', 'ICN', 
            'JFK', 'LGA', 'LHR', 'ORD', 'SAN', 'SFO', 'SIN', 'TLV', 'BUD']

In [337]:
def minAdditionalRoutes(startingAirport, routes, airports):
    
    def findRecursivePaths(_airport, path_dict, \
                           paths=set()):
        if _airport not in path_dict:
            return paths
        _paths = [a for a in path_dict[_airport]]
        for _path in _paths:
            # stop when cyclical
            if _path in paths:
                break
            paths.add(_path)
            findRecursivePaths(_path, path_dict, \
                                 paths=paths)
        return paths
            
        
    min_paths = 0
    
    # deduplicate
    airports = set(airports)
    routes = list(set(tuple(row) for row in routes))
    
    # base case
    if startingAirport not in airports:
        return 'startingAirport is not in airports'
    
    # first degree upstream and downstream connections from each airport
    upstream_dict, downstream_dict = dict(), dict()
    for d in airports:
        upstream_dict[d] = set()
        downstream_dict[d] = set()
        for r in routes:
            if r[1] == d:
                upstream_dict[d].add(r[0])
            if r[0] == d:
                downstream_dict[d].add(r[1])
    
    r_upstream_dict, r_downstream_dict = dict(), dict()
    
    for airport in airports:
        upstream_paths = findRecursivePaths(airport, 
                                              upstream_dict, 
                                              paths=set())
        
        upstream_paths = sorted(list(upstream_paths))

        r_upstream_dict[airport] = upstream_paths

        downstream_paths = findRecursivePaths(airport, 
                                                downstream_dict, 
                                                paths=set())
    
        downstream_paths = sorted(list(downstream_paths))
        r_downstream_dict[airport] = downstream_paths

    missing = set(airports).difference(set(r_downstream_dict[startingAirport]))

    if len(missing) == 0:
        return min_paths
    
    for k,v in r_upstream_dict.items():
        if v == []:
            min_paths += 1
            missing.remove(k)
            missing = missing.difference(set(r_downstream_dict[k]))
      
    downstream_list = [r_downstream_dict[miss] for miss in missing]
        
    downstream_set = [list(y) for y in set(tuple(x) for x in downstream_list)]
    downstream_set.sort(key=lambda x:len(x), reverse=True)
    
    for _downstream_set in downstream_set:
        min_paths += 1
        missing = missing.difference(set(_downstream_set))
        if len(missing) == 0:
            return min_paths
    return min_paths

# tests
assert minAdditionalRoutes('LGA', routes, airports) == 3
assert minAdditionalRoutes('EYW', routes, airports) == 2
assert minAdditionalRoutes('DSM', routes, airports) == 3
assert minAdditionalRoutes('SFO', routes, airports) == 2